# League Table

We will create league table from the all individual matches data in a season

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#This next line makes our charts show up in the notebook
%matplotlib inline

df = pd.read_csv("season-1213.csv")
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,E0,18/08/12,Arsenal,Sunderland,0,0,D,0,0,D,...,2.05,23,-1.25,2.02,1.96,1.96,1.91,1.44,4.72,8.71
1,E0,18/08/12,Fulham,Norwich,5,0,H,2,0,H,...,1.92,21,-0.50,1.83,1.80,2.14,2.09,1.84,3.75,4.75
2,E0,18/08/12,Newcastle,Tottenham,2,1,H,0,0,D,...,1.92,23,0.00,1.93,1.88,2.03,1.97,2.83,3.35,2.72
3,E0,18/08/12,QPR,Swansea,0,5,A,0,1,A,...,1.73,22,-0.25,1.80,1.75,2.21,2.13,2.00,3.53,4.15
4,E0,18/08/12,Reading,Stoke,1,1,D,0,1,A,...,1.67,24,-0.25,2.07,2.01,1.91,1.86,2.47,3.30,3.22


In [15]:
droplist = list(df.columns)
droplist = droplist[droplist.index('HTHG'):]
df.drop(droplist, axis = 1, inplace = True)
df.drop(['Div','Date'],axis=1,inplace=True)

df.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,Arsenal,Sunderland,0,0,D
1,Fulham,Norwich,5,0,H
2,Newcastle,Tottenham,2,1,H
3,QPR,Swansea,0,5,A
4,Reading,Stoke,1,1,D


In [16]:
#Wins
#Home Team
df['HW'] = np.where(df['FTR']=='H', 1, 0)
df['HD'] = np.where(df['FTR']=='D', 1, 0)
df['HL'] = np.where(df['FTR']=='A', 1, 0)
#Away Team
df['AW'] = np.where(df['FTR']=='A', 1, 0)
df['AD'] = np.where(df['FTR']=='D', 1, 0)
df['AL'] = np.where(df['FTR']=='H', 1, 0)

#Points
conditions = [df['FTR']=='H',df['FTR']=='D']
home_points = [3,1]
away_points = [0,1]
df['HP'] = np.select(conditions, home_points, default=0) #Home Points
df['AP'] = np.select(conditions, away_points, default=3) #Away Points
#There's more effective way to add points by using formula later after the table has been grouped by teams,
#but for educational reason, we use that way to demonstrate how to add conditional columns with more than 2 choices

#Show Data
df.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HW,HD,HL,AW,AD,AL,HP,AP
0,Arsenal,Sunderland,0,0,D,0,1,0,0,1,0,1,1
1,Fulham,Norwich,5,0,H,1,0,0,0,0,1,3,0
2,Newcastle,Tottenham,2,1,H,1,0,0,0,0,1,3,0
3,QPR,Swansea,0,5,A,0,0,1,1,0,0,0,3
4,Reading,Stoke,1,1,D,0,1,0,0,1,0,1,1


In [25]:
df_home = df.groupby('HomeTeam').sum()
#HomeGamesPlayed
df_home['HGP'] = df_home['HW'] + df_home['HD'] + df_home['HL']

#Renaming goals scored and against
df_home = df_home.rename(columns = {'FTHG':'HGS'})
df_home = df_home.rename(columns = {'FTAG':'HGA'})
#HomeGoalDifference
df_home['HGD'] = df_home['HGS'] - df_home['HGA']
df_home[['HGP','HP','HW','HD','HL','HGS','HGA','HGD']].sort_values(by=['HP'],ascending=False)

,HGP,HP,HW,HD,HL,HGS,HGA,HGD
HomeTeam,,,,,,,,
Man United,19,48,16,0,3,45,19,26
Man City,19,45,14,3,2,41,15,26
Everton,19,42,12,6,1,33,17,16
Chelsea,19,41,12,5,2,41,16,25
Arsenal,19,38,11,5,3,47,23,24
Tottenham,19,38,11,5,3,29,18,11
West Ham,19,33,9,6,4,34,22,12
Liverpool,19,33,9,6,4,33,16,17
Norwich,19,31,8,7,4,25,20,5


Above is the league table for home games

In [24]:
df_away = df.groupby('AwayTeam').sum()
#HomeGamesPlayed
df_away['AGP'] = df_away['AW'] + df_away['AD'] + df_away['AL']

#Renaming goals scored and against
df_away=df_away.rename(columns = {'FTHG':'AGA'})
df_away=df_away.rename(columns = {'FTAG':'AGS'})
#HomeGoalDifference
df_away['AGD'] = df_away['AGS'] - df_away['AGA']
df_away.drop(columns = ['HW','HD','HL','HP'], inplace=True)
df_away[['AGP','AP','AW','AD','AL','AGS','AGA','AGD']].sort_values(by=['AP'],ascending=False)

,AGA,AGS,AW,AD,AL,AP,AGP,AGD
AwayTeam,,,,,,,,
Arsenal,14,25,10,5,4,35,19,11
Aston Villa,41,24,5,6,8,21,19,-17
Chelsea,23,34,10,4,5,34,19,11
Everton,23,22,4,9,6,21,19,-1
Fulham,30,22,4,7,8,19,19,-8
Liverpool,27,38,7,7,5,28,19,11
Man City,19,25,9,6,4,33,19,6
Man United,24,41,12,5,2,41,19,17
Newcastle,37,21,2,7,10,13,19,-16


Above is the league table for away games

In [21]:
#Create the league table
league_table=df_home[[]]
league_table['GP']=df_home['HGP']+df_away['AGP']
league_table['Points']=df_home['HP']+df_away['AP']
league_table['W']=df_home['HW']+df_away['AW']
league_table['D']=df_home['HD']+df_away['AD']
league_table['L']=df_home['HL']+df_away['AL']
league_table['GS']=df_home['HGS']+df_away['AGS']
league_table['GA']=df_home['HGA']+df_away['AGA']
league_table['GD']=df_home['HGD']+df_away['AGD']
league_table.sort_values(by=['Points'],ascending=False,inplace=True)
league_table

C:\Users\Thyo\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Thyo\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Thyo\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,GP,Points,W,D,L,GS,GA,GD
HomeTeam,,,,,,,,
Man United,38,89,28,5,5,86,43,43
Man City,38,78,23,9,6,66,34,32
Chelsea,38,75,22,9,7,75,39,36
Arsenal,38,73,21,10,7,72,37,35
Tottenham,38,72,21,9,8,66,46,20
Everton,38,63,16,15,7,55,40,15
Liverpool,38,61,16,13,9,71,43,28
West Brom,38,49,14,7,17,53,57,-4
West Ham,38,46,12,10,16,45,53,-8


We have created the league table. Now if you want to see the League table for the last 3 seasons combined, for example, you can do this:

In [22]:
#ToBeContinued